In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/SimCSE-main")

Mounted at /content/drive


In [2]:
!pip install torch==1.7.1
!pip install -r /content/drive/MyDrive/SimCSE-main/requirements.txt
!pip install dill==0.3.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 776.8 MB 14 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.1 MB/s 
     |████████████████████████████████| 25.9 MB 35.1 MB/s 
  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 86 kB 3.1 MB/s 
  Attempting uninstall: dill
    Found existing installation: dill 0.3.5.1
    Uninstalling dill-0.3.5.1:
      Successfully uninstalled dill-0.3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.13 requires dill>=0.3.5.1, but you have dill 0.3.4 which is incompatible.


In [ ]:
!bash run_sup_example.sh

07/10/2022 13:03:30 - INFO - __main__ -   PyTorch: setting up devices
model_ards12 ModelArguments(model_name_or_path='bert-base-uncased', model_type=None, config_name=None, tokenizer_name=None, cache_dir=None, use_fast_tokenizer=True, model_revision='main', use_auth_token=False, temp=0.05, pooler_type='cls', hard_negative_weight=0, do_mlm=False, mlm_weight=0.1, mlp_only_train=False)
07/10/2022 13:03:30 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1 distributed training: False, 16-bits training: True
07/10/2022 13:03:30 - INFO - __main__ -   Training/evaluation parameters OurTrainingArguments(output_dir='/content/drive/MyDrive/SimCSE-main/result/c5_ep10_bert_full', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=128, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_s

In [ ]:
!python simcse_to_huggingface.py --path result/compare/c5_ep10_bert_full

SimCSE checkpoint -> Huggingface checkpoint for result/compare/c5_ep10_bert_full


In [4]:
#  finetuned model produce embedding of test data
#!pip install transformers==4.10.1
from transformers import AutoTokenizer,AutoModel
from tqdm import tqdm
import os
import time, torch, json

device = torch.device("cuda" if torch.cuda.is_available()  else "cpu")
print(device)

import pandas as pd

def extract_all_samples(dataset):
    samples = []
    utterances = []
    personas = []
    for d in dataset:
        ua = [u['A'][0] for u in d['dialogue']]
        ub = [u['B'][0] for u in d['dialogue']]
        pa = [p for _, p in d['pa'].items() if not pd.isnull(p) and len(p) > 0]  # a=" " len(a)=1
        pb = [p for _, p in d['pb'].items() if not pd.isnull(p) and len(p) > 0]
        samples.append((ua, pa))
        samples.append((ub, pb))

        utterances += ua + ub
        personas += pa + pb


    return samples, utterances, personas 

lm='bert'
input='/content/drive/MyDrive/SimCSE-main/persona_linking_test.json'
output='/content/drive/MyDrive/SimCSE-main/result/workable_models/c5_ep10_bert_full.bert'
# 将原文本用bert预处理为bert embedding, 生成bert文件

if lm == 'bert':

    tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/SimCSE-main/result/compare/c5_ep10_bert_full')
    model = AutoModel.from_pretrained('/content/drive/MyDrive/SimCSE-main/result/compare/c5_ep10_bert_full')
    
    model.to(device)

    

with open(input, 'r') as f:
    # data preparation
    train_dataset = json.load(f)
    samples_text, utterances, personas = extract_all_samples(train_dataset)
  

    print('dialogues[{}], utterances[{}], personas[{}]'.format(
        len(samples_text) / 2, len(utterances), len(personas)))

u_d = []
p_d = []

b_time = time.time()

for i in tqdm(range(len(samples_text))):  #显示处理到第几条数据
    #print('i',i)
    u_s = samples_text[i][0]
    p_s = samples_text[i][1]
   

    u_tmp = torch.stack([model(tokenizer(u_t,max_length=25,padding=True,truncation=True,return_tensors='pt').to(device),output_hidden_states=True, return_dict=True).pooler_output for u_t in u_s], dim = 0)
    u_tmp = torch.stack([model(tokenizer.encode(p_t,max_length=25,padding=True,truncation=True,return_tensors='pt').to(device)).pooler_output for p_t in p_s], dim = 0)
    
    u_d.append(u_tmp.data) #某个人的话语和对应的个人信息描述
    p_d.append(p_tmp.data)

torch.save({'utterance': u_d, 'persona': p_d}, output)

e_time = time.time()

print('time: {}'.format(e_time - b_time))



cpu


RuntimeError: ignored